In [2]:
import os
import numpy as np
# import pandas as pd
from tqdm import tqdm
# from PIL import Image, ImageDraw
import glob
# import matplotlib.pyplot as plt
# import time
import tensorflow_hub as hub
import tensorflow as tf
import cv2


from operator import itemgetter
from collections import OrderedDict
tf.compat.v1.disable_eager_execution()

tf.__version__
import gc
gc.enable()
cwd = os.getcwd()
output_dir = os.path.join(cwd, "../drawn_boxes/")

In [ ]:
def draw_boxes(path, filename, im, result):
    
    print(filename)
    # draw = ImageDraw.Draw(im)
    im_height, im_width, channels = im.shape
    detection_count = 0
    os.chdir(output_dir)
    for index, box in enumerate(result['detection_boxes'][:5]):
        if result['detection_classes'][index] == 1 and result['detection_scores'][index] >= 0.85:
            detection_count += 1
            # (left, right, top, bottom) = (box[0]*im_width, box[1]*im_width, box[2]*im_height, box[3]*im_height)

            (left, right) = tuple(im_width*np.array(
                (box[1], box[0])
                if box[0] > box[1]
                else (box[0], box[1])
            ))
            # (left, right) = (left*im_width, right*im_width)
            (top, bottom) = tuple(im_height*np.array(
                (box[3], box[2]) 
                if box[2] > box[3] 
                else (box[2], box[3])
            ))
            # (top, bottom) = (top*im_height, bottom*im_height)


            cv2.rectangle(im, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)
            # draw.line([(left, top), (left, bottom), (right, bottom),
            #        (right, top), (left, top)], width=4, fill='Red')
            print(path, "\n", (im.shape), "\n", (box[0], box[1], box[2], box[3]), (left, right, top, bottom), (int(left), int(right), int(top), int(bottom)))
            # cropped = cv2.imread(im[int(top):int(bottom)])

            print(im[int(top):int(bottom), int(left):int(right),:].shape)
            cv2.imshow('full',im)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.imshow('cropped',im[int(top):int(bottom), int(left):int(right), :])
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            # cropped = im.crop((left, top, right, bottom))
            # cropped.save("".join(["../drawn_boxes/", filename.split(".")[0],"_cropped", str(detection_count), ".jpg"]))
            cv2.imwrite("".join([filename.split(".")[0],"_cropped", str(detection_count), ".jpg"]), im[int(top):int(bottom), int(left):int(right)])
            # cropped.load()

        if detection_count > 0:
            # im.save("".join(["../drawn_boxes/", filename]))
            cv2.imwrite("".join([filename.split(".")[0], ".jpg"]), im)

        '''Trying to clear out some memory'''
        left = right = top = bottom = cropped = None
        del(left, right, top, bottom, cropped)
    detection_count = im_width = im_height = draw = im = None
    del(detection_count, im_width, im_height, draw, path, filename, im, result)
    os.chdir(cwd)
    

In [ ]:
module = hub.Module("https://tfhub.dev/microsoft-ai-for-earth/vision/detector/megadetector_V3/1")
height, width = hub.get_expected_image_size(module)
print(hub.get_num_image_channels(module))
print(os.getcwd())
# path_to_files = os.path.join(os.getcwd(), os.path., 'subdir', 'dir', 'filename.foo')
# print(module.get_tensor_by_name('image_tensor:0'))
print(height, width)
results = []
images = []
sess = tf.compat.v1.Session()
testset = np.zeros((len(glob.glob("..\\Dataset\\test\\*.jpg")), height, width, 3))
for i in tqdm(range(0, 290, 10)):
    files = glob.glob("../Dataset/test/*.jpg")[i:i+10]
    for file in tqdm(files):
        sess = tf.compat.v1.Session()
        filename = file.split("/")[-1:][0]
        testset = np.zeros((1, height, width, 3))
        print(file)
        img = cv2.resize(cv2.imread(file),(width,height))
        print(type(img))
        testset[i,:,:,:] = img
        # output = module(testset, as_dict=True)
        # output_dict = {
        #     'detection_scores': output['detection_scores'].eval(session=sess)[0],
        #     'detection_classes': output['detection_classes'].eval(session=sess)[0],
        #     'detection_boxes': output['detection_boxes'].eval(session=sess)[0]
        # }
        # output_dict = OrderedDict(sorted(output_dict.items(), key=lambda x: x[0], reverse=True))
        # draw_boxes(file, filename, img, output_dict)
        '''Trying to clear out some memory'''
        utput_dict = img = testset = filename = sess = None


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
3
/home/job/project_aai/python/megadetector_V3
512 512
../Dataset/test/8a0e92b0-21bc-11ea-a13a-137349068a90.jpg
<class 'numpy.ndarray'>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
8a0e92b0-21bc-11ea-a13a-137349068a90.jpg
../Dataset/test/8a2b4626-21bc-11ea-a13a-137349068a90.jpg
<class 'numpy.ndarray'>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
8a2b4626-21bc-11ea-a13a-137349068a90.jpg
../Dataset/test/8a2b4626-21bc-11ea-a13a-137349068a90.jpg 
 (512, 512, 3) 
 (0.027220702, 0.3717098, 0.18696682, 0.5277829) (13.936999, 190.31541, 95.72701263427734, 270.224853515625) (13, 190, 95, 270)
(175, 177, 3)
../Dataset/test/8a2ad0a6-21bc-11ea-a13a-137349068a90.jpg
<class 'numpy.ndarray'>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
8a2ad0a6-21bc-11ea-a13a-137349068a90.jpg
../Data

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
  0%|          | 0/10 [00:00<?, ?it/s]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 10%|█         | 1/10 [00:26<03:59, 26.59s/it]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 20%|██        | 2/10 [00:55<03:38, 27.35s/it]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 30%|███       | 3/10 [01:25<03:15, 27.98s/it]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 40%|████      | 4/10 [01:55<02:51, 28.58s/it]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 50%|█████     | 5/10 [02:25<02:26, 29.24s/it]INFO:tensorflow:Saver not created because there are no variables in the graph to restore

 60%|██████    | 6/10 [02:57<02:00, 30.08s/it]INFO:tensorflow:Saver not created because there are no va